In [6]:
import math
import numpy as np
import pandas
import itertools
from sklearn.metrics import precision_score, roc_auc_score


uii_data = pandas.read_csv('../working_data/updrsii_all_q_p_s.csv')

In [2]:
uii_all_qs = {'D_NP2SPCH', 'D_NP2SALV', 'D_NP2SWAL', 'D_NP2EAT',
       'D_NP2DRES', 'D_NP2HYGN', 'D_NP2HWRT', 'D_NP2HOBB', 'D_NP2TURN',
       'D_NP2TRMR', 'D_NP2RISE', 'D_NP2WALK', 'D_NP2FREZ', 'D_NP2PTOT'}

uii_sets = []

for x in range(1,len(uii_all_qs)):
    uii_sets.append(list(itertools.combinations(uii_all_qs, x)))

In [3]:
def query_builder(qs, threshold):
    output = ""
    for q in qs:
        add = f'{q} > {threshold} and '
        output = output + add        
    return output[:-5]

def anti_query_builder(qs,threshold):
    output = ""
    for q in qs:
        add = f'{q} <= {threshold} and '
        output = output + add        
    return output[:-5]

def test_q_subset(source, qs, threshold, results):
    df_positive = source.query(query_builder(qs, threshold))
    df_negative = source.query(anti_query_builder(qs,threshold))
    row,_ = df_positive.shape
    if row > 5:
        y1 = df_positive['SIGFALL'].to_list()
        y2 = df_negative['SIGFALL'].to_list()
        y = y1 + y2
        predictions1 = [1.0] * len(y1)
        predictions2 = [0.0] * len(y2)
        predictions = predictions1 + predictions2
        p = precision_score(y, predictions, zero_division=0)
        roc = roc_auc_score(y, predictions)
        if p > 0.49:
            results.append((qs,{'roc-auc':roc,'precision':p}))
        else: 
            if roc > 0.6:
                results.append((qs,{'roc-auc':roc,'precision':p}))

In [4]:
uii_flatten = [element for sublist in uii_sets for element in sublist]
uii_flatten[0]
len(uii_flatten)

16382

In [7]:
uii_0_results = []
for qs in uii_flatten:
    test_q_subset(uii_data, qs,0, uii_0_results)
    
uii_0_results

[(('D_NP2EAT', 'D_NP2FREZ', 'D_NP2HWRT'),
  {'roc-auc': 0.529160419790105, 'precision': 0.5}),
 (('D_NP2FREZ', 'D_NP2SWAL', 'D_NP2HWRT'),
  {'roc-auc': 0.5569804721655494, 'precision': 0.5}),
 (('D_NP2EAT', 'D_NP2FREZ', 'D_NP2HYGN', 'D_NP2DRES'),
  {'roc-auc': 0.529985119047619, 'precision': 0.5}),
 (('D_NP2EAT', 'D_NP2FREZ', 'D_NP2HYGN', 'D_NP2HOBB'),
  {'roc-auc': 0.5339622641509434, 'precision': 0.5}),
 (('D_NP2EAT', 'D_NP2FREZ', 'D_NP2HWRT', 'D_NP2PTOT'),
  {'roc-auc': 0.5454268292682928, 'precision': 0.5}),
 (('D_NP2SALV', 'D_NP2FREZ', 'D_NP2TURN', 'D_NP2HOBB'),
  {'roc-auc': 0.533445945945946, 'precision': 0.5}),
 (('D_NP2FREZ', 'D_NP2WALK', 'D_NP2RISE', 'D_NP2TRMR'),
  {'roc-auc': 0.5445111277819454, 'precision': 0.5}),
 (('D_NP2FREZ', 'D_NP2SWAL', 'D_NP2HWRT', 'D_NP2HOBB'),
  {'roc-auc': 0.5473821989528797, 'precision': 0.6666666666666666}),
 (('D_NP2EAT', 'D_NP2FREZ', 'D_NP2HYGN', 'D_NP2PTOT', 'D_NP2DRES'),
  {'roc-auc': 0.5455882352941177, 'precision': 0.5}),
 (('D_NP2EAT', '

In [8]:
a,b = uii_0_results[0]
list(a)
z = []

for x in uii_0_results:
    a,_ = x
    z.append(list(a))
    
flatten_list = [element for sublist in z for element in sublist]
flatten_list

(unique, counts) = np.unique(flatten_list, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(len(z))
frequencies

12


array([['D_NP2DRES', '2'],
       ['D_NP2EAT', '6'],
       ['D_NP2FREZ', '12'],
       ['D_NP2HOBB', '5'],
       ['D_NP2HWRT', '4'],
       ['D_NP2HYGN', '4'],
       ['D_NP2PTOT', '5'],
       ['D_NP2RISE', '2'],
       ['D_NP2SALV', '2'],
       ['D_NP2SWAL', '2'],
       ['D_NP2TRMR', '2'],
       ['D_NP2TURN', '2'],
       ['D_NP2WALK', '2']], dtype='<U21')

In [9]:
uii_1_results = []
for qs in uii_flatten:
    test_q_subset(uii_data, qs,1, uii_1_results)
    
uii_1_results

[]

In [36]:
uii_2_results = []
for qs in uii_flatten:
    test_q_subset(uii_data, qs,2, uii_2_results)
    
uii_2_results

[(('D_NP2FREZ',), 0.42857142857142855),
 (('D_NP2PTOT', 'D_NP2FREZ'), 0.42857142857142855)]

In [37]:
uii_3_results = []
for qs in uii_flatten:
    test_q_subset(uii_data, qs,3, uii_3_results)
    
uii_3_results

[]

In [70]:
df_univ = uii_data.query("D_NP2RISE > 0 and D_NP2WALK > 0 and D_NP2FREZ > 0 and D_NP2PTOT >0")
print(df_univ.shape)

y = df_univ['SIGFALL']
predictions = [1.0] * len(y)
precision_score(y, predictions, zero_division=0)

(15, 16)


0.3333333333333333

In [68]:
df_univ

,D_NP2SPCH,D_NP2SALV,D_NP2SWAL,D_NP2EAT,D_NP2DRES,D_NP2HYGN,D_NP2HWRT,D_NP2HOBB,D_NP2TURN,D_NP2TRMR,D_NP2RISE,D_NP2WALK,D_NP2FREZ,D_NP2PTOT,PREV_SIGFALL,SIGFALL
74,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,5.0,0.0,1.0
133,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,7.0,0.0,0.0
147,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,1.0,0.0,-1.0,1.0,1.0,1.0,3.0,0.0,0.0
180,-1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,4.0,1.0,0.0
181,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,16.0,0.0,0.0
201,0.0,3.0,0.0,2.0,2.0,2.0,0.0,0.0,1.0,4.0,2.0,2.0,2.0,20.0,0.0,0.0
246,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.0,1.0,1.0,1.0,8.0,0.0,1.0
310,3.0,3.0,0.0,0.0,2.0,0.0,1.0,2.0,1.0,0.0,2.0,2.0,3.0,19.0,0.0,0.0
338,0.0,-2.0,0.0,0.0,1.0,-1.0,-1.0,2.0,1.0,0.0,1.0,1.0,2.0,4.0,0.0,0.0
367,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,0.0,0.0
